In [ ]:




#Energy Coversing Orientational - Driving Force  (ECO-DF)





In [1]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from ase import Atoms
from ase.io import read, write
from ase.build import surface, cut
from ase.optimize import BFGS
from ase.constraints import UnitCellFilter
from ase.visualize import view
import pyace
#import aimsgb
#from aimsgb import GrainBoundary, Grain
import os
import shutil
import subprocess
from scipy.interpolate import griddata
from ase.constraints import FixedLine

In [2]:
pot = pyace.PyACECalculator("/nfshome/karanikv/TEST_Potential/output_potential.yaml")
ASI = "/nfshome/karanikv/TEST_Potential/output_potential.asi"
pot.set_active_set(ASI)

In [3]:
def gb_energy(atoms, eref_Cu, eref_Ag, xAg=0):
    a, b = atoms.cell[0], atoms.cell[1]
    A = np.linalg.norm(np.cross(a,b))
    e = atoms.get_potential_energy()
    return (e - len(atoms) *((1 - xAg) * eref_Cu + xAg * eref_Ag)) / (2*A)

def optimize(atoms, fmax=0.1, write=False, fname=None):
    # Constrain each atom individually along the z-direction
    indices1 = [atom.index for atom in atoms]
    #c = FixedLine(indices=indices, direction=[0, 0, 1])
    indices = list(range(len(atoms)))  # Ensure indices is a list of atom indices
    CC = []
    for index in indices:
        CC.append(FixedLine(index, [0, 0, 1]))
    atoms.set_constraint(CC)

    # Initialize and run the BFGS optimizer
    opt = BFGS(atoms)
    opt.run(fmax=fmax,steps=250)

    # Optionally write the optimized structure to a file
    if write:
        if fname:
            atoms.write("{}".format(fname), vasp5=True, direct=True, sort=True)
        else:
            atoms.write("opt.POSCAR", vasp5=True, direct=True, sort=True)

    return None

def optimize1(atoms, fmax=0.5, mask=(1, 1, 1, 0, 0, 0), write=False, fname=None):
    opt = BFGS(atoms)
    opt.run(fmax=fmax,steps=250)
    ucf = UnitCellFilter(atoms,constant_volume=True, mask=mask)
    opt = BFGS(ucf)
    opt.run(fmax=fmax,steps=250)
    if write:
        if fname:
            atoms.write("{}".format(fname), vasp5=True, direct=True, sort=True)
        else:
            atoms.write("opt.POSCAR", vasp5=True, direct=True, sort=True)

    return None

def make_cell_diagonal(atoms):
    # Get the cell matrix
    cell = atoms.get_cell()

    # Compute the transformation matrix that makes the cell diagonal
    # Use Singular Value Decomposition (SVD)
    U, _, Vt = np.linalg.svd(cell)
    transformation_matrix = np.dot(U, Vt)

    # Apply the transformation to the cell
    new_cell = np.dot(cell, transformation_matrix.T)
    atoms.set_cell(new_cell, scale_atoms=False)

    # Apply the transformation to the atomic positions
    new_positions = np.dot(atoms.get_positions(), transformation_matrix.T)
    atoms.set_positions(new_positions)

    return atoms

def make_cell_tetragonal(atoms):
    # Get the cell matrix
    cell = atoms.get_cell()

    # Compute the transformation matrix that makes the cell diagonal
    U, _, Vt = np.linalg.svd(cell)
    transformation_matrix = np.dot(U, Vt)

    # Apply the transformation to the cell
    new_cell = np.dot(cell, transformation_matrix.T)

    # Ensure the new cell is diagonal
    new_cell = np.diag(np.diag(new_cell))

    # Make the cell tetragonal: Set a and b equal, c can be different
    a = (new_cell[0, 0] + new_cell[1, 1]) / 2
    new_cell[0, 0] = a
    new_cell[1, 1] = a

    # Apply the transformation to the atomic positions
    atoms.set_cell(new_cell, scale_atoms=False)
    new_positions = np.dot(atoms.get_positions(), transformation_matrix.T)
    atoms.set_positions(new_positions)

    return atoms